In [1]:
import pandas as pd
import numpy as np
import math
from collections import defaultdict
import random as rand
from sklearn.metrics.pairwise import euclidean_distances
from scipy.stats import multivariate_normal
import mnist_reader
import csv

In [2]:
def log_sum_exp(u,v):
    return (max(u, v) + math.log(math.exp(u - max(u, v)) + math.exp(v - max(u, v))))

def logpdf(data,meu , sigma):
    logpdfs=[]
    for i in data:
        val = 0.0
        dim = len(data[0])
        val+= (math.log(2*math.pi)*-0.5*dim)
        
        #Cholesky det
        #logdet = np.linalg.cholesky(sigma).diagonal()
        #logdet = [math.log(i) for i in logdet]
        #val+= (-0.5*2*sum(logdet))
        
        val+= (math.log(np.linalg.det(sigma))*-0.5)
        mat = np.array(i)-meu
        val+= (-0.5*np.matmul(np.matmul(mat.T,np.linalg.inv(sigma)),mat))
        
        logpdfs.append(val)
    
    return logpdfs

def Estep(k,data,meuMatrix,sigMatrix,wMatrix,piMatrix):
    pdfMatrix=[]
    logpdfMatrix=[]
    for i in range(k):    
        #pdfMatrix.append(multivariate_normal.pdf(data,meuMatrix[i], sigMatrix[i]))
        if np.linalg.det(sigMatrix[i])<1e-60:
            for j in range(len(sigMatrix[i])):
                sigMatrix[i][j][j]+=0.0001
                
        logpdfMatrix.append(logpdf(data,meuMatrix[i], sigMatrix[i]))
        
    
    for i in range(len(data)):
        for j in range(k):
            piMatrix[i][j] = float(logpdfMatrix[j][i])+math.log(wMatrix[j])
    
        denom=log_sum_exp(logpdfMatrix[0][i]+math.log(wMatrix[0]), logpdfMatrix[1][i]+math.log(wMatrix[1]))
        for j in range(2,k):
            denom = log_sum_exp(denom,logpdfMatrix[j][i]+math.log(wMatrix[j]))

        for j in range(k):
            piMatrix[i][j] = math.exp(float(piMatrix[i][j]) - denom)
    
    '''
    for i in range(len(data)):
        sum1=0
        for j in range(k):
            piMatrix[i][j] = float(pdfMatrix[j][i])*(wMatrix[j])
            sum1+=float(pdfMatrix[j][i])*(wMatrix[j])

        for j in range(k):
            piMatrix[i][j] = float(piMatrix[i][j])/sum1
    '''   
    return (piMatrix,logpdfMatrix)

def Mstep(k,data,piMatrix):
    
    wMatrix=[]
    for i in range(k):
        wMatrix.append(float(sum(pd.DataFrame(piMatrix)[i]))/float(len(data)))
    
    meuMatrix = []
    for i in range(k):
        numer=np.array(piMatrix[0][i])*data[0]
        denom=piMatrix[0][i]
        for j in range(1,len(data)):
            numer += np.array(piMatrix[j][i])*data[j]
            denom += piMatrix[j][i]
        
        meuMatrix.append(np.array(numer)/float(denom))
    
    sigMatrix=[]
    for i in range(k):
        temp = np.array(data[0]-meuMatrix[i])[np.newaxis]
        numer = piMatrix[0][i]*np.matmul(temp.T,temp)
        denom = piMatrix[0][i]
        for j in range(1,len(data)):
            temp = np.array(data[j]-meuMatrix[i])[np.newaxis]
            numer += piMatrix[j][i]*np.matmul(temp.T,temp)
            denom += piMatrix[j][i]
        sigMatrix.append(np.array(numer)/float(denom))
    
    return (np.array(meuMatrix),np.array(sigMatrix),wMatrix)

In [3]:
def GaussianMixture(k, data):
    meuMatrix=[]        
    for i in range(k):
        meuMatrix.append(data[rand.randint(0,len(data)-1)])  
        
    piMatrix =[]
    for i in range(len(data)):
        piMatrix.append([0 for j in range(k)])
        
    sim = euclidean_distances(data,np.array(meuMatrix))
    for i in range(len(sim)):
        ind = np.unravel_index(np.argmin(sim[i], axis=None), sim[i].shape)
        piMatrix[i][ind[0]]=1
    
    
    ic=0
    prevlikelihood = 0
    while(True):
        (meuMatrix,sigMatrix,wMatrix) = Mstep(k,data,piMatrix)
        
        (piMatrix,logpdfMatrix) = Estep(k,data,meuMatrix,sigMatrix,wMatrix,piMatrix)
        
        likelihood=0
        for i in range(len(logpdfMatrix)):
            for j in range(len(logpdfMatrix[i])):
                likelihood+= piMatrix[j][i]*logpdfMatrix[i][j] + piMatrix[j][i]*math.log(wMatrix[i])
        
        ic+=1
        print "Iteration",ic,"Log-Likelihood:",likelihood
        
        if abs(prevlikelihood-likelihood)>1e-4:
            prevlikelihood = likelihood
        else:
            break
        
    return piMatrix

In [4]:
def performance_calculator(cluster,categories):
    confusion_matrix=[]
    for i in cluster:
        countDict=defaultdict(int)
        for j in cluster[i]:
            countDict[j[1]]+=1
    
        confusion_matrix.append([countDict[i] for i in range(categories)])
    
    numerator = 0
    denominator = 0
    for i in confusion_matrix:
        numerator+=max(i)
        denominator+=sum(i)
    
    purity = float(numerator)/float(denominator)
    
    gini_indexes=[]
    total=0
    for i in confusion_matrix:
        gi=1
        for j in i:
            gi-=((float(j)/float(sum(i)))**2)
        gini_indexes.append(gi*sum(i))
        total+=sum(i)
        
    gini = float(sum(gini_indexes))/float(total)
    
    return (purity,gini,confusion_matrix)

def GaussianEval(k,piMatrix,data,labels):
    cluster={}
    for i in range(k):
        cluster[i]=[]
    
    for i in range(len(piMatrix)):
        ind = np.unravel_index(np.argmax(piMatrix[i], axis=None), np.array(piMatrix[i]).shape)
        
        cluster[ind[0]].append((data[i],labels[i]))
    
    print "Gaussian Mixtures: ",k
    
    purity,gini,confusion_matrix = performance_calculator(cluster,len(set(labels)))
    
    print "Purity: ",purity
    print "Gini: ",gini
    print "Confusion Matrix: ",display(pd.DataFrame(confusion_matrix))


In [5]:
def load_spam_data():
    data = []

    f = open('/Users/sasankauppu/Desktop/Data Mining CS6220/DataMining/spambase/spambase.data')
    reader = csv.reader(f)
    
    for row in reader:
        data.append(row)
    f.close()
    
    X = np.array([x[:-1] for x in data]).astype(np.float)
    y = np.array([x[-1] for x in data]).astype(np.float)

    return X, y

In [8]:
spam_X,spam_y = load_spam_data()

In [9]:
piMatrix=GaussianMixture(2,spam_X)

Iteration 1 Log-Likelihood: -150879.73732
Iteration 2 Log-Likelihood: -101596.015659
Iteration 3 Log-Likelihood: -77356.2754597
Iteration 4 Log-Likelihood: -62555.647036
Iteration 5 Log-Likelihood: -54144.8392705
Iteration 6 Log-Likelihood: -51817.8851371
Iteration 7 Log-Likelihood: -49479.6310961
Iteration 8 Log-Likelihood: -48219.9849703
Iteration 9 Log-Likelihood: -47584.7936273
Iteration 10 Log-Likelihood: -47285.8470775
Iteration 11 Log-Likelihood: -47026.532508
Iteration 12 Log-Likelihood: -46782.761827
Iteration 13 Log-Likelihood: -46615.1430215
Iteration 14 Log-Likelihood: -46398.3295946
Iteration 15 Log-Likelihood: -46226.6884386
Iteration 16 Log-Likelihood: -46174.4575956
Iteration 17 Log-Likelihood: -46114.9430364
Iteration 18 Log-Likelihood: -46032.183149
Iteration 19 Log-Likelihood: -45914.4334646
Iteration 20 Log-Likelihood: -45801.812919
Iteration 21 Log-Likelihood: -45678.5947795
Iteration 22 Log-Likelihood: -45576.4838989
Iteration 23 Log-Likelihood: -45498.8371866
Ite

In [10]:
GaussianEval(2,piMatrix,spam_X,spam_y)

Gaussian Mixtures:  2
Purity:  0.779830471637
Gini:  0.313180842811
Confusion Matrix: 

,0,1
0,819,1619
1,1969,194


 None


In [12]:
X_train, y_train = mnist_reader.load_mnist('data/', kind='train')
X_test, y_test = mnist_reader.load_mnist('data/', kind='t10k')

fashion_X = np.concatenate((X_train,X_test))
fashion_y = np.concatenate((y_train,y_test))

del(X_train)
del(X_test)
del(y_train)
del(y_test)

In [ ]:
#piMatrix=GaussianMixture(10,fashion_X)

In [17]:
from sklearn.mixture import GaussianMixture
gmm = GaussianMixture(n_components=10,tol=1e-4,max_iter=300,verbose=True).fit(fashion_X)
labels = gmm.predict(fashion_X)

Initialization 0
  Iteration 0
  Iteration 10
  Iteration 20
  Iteration 30
  Iteration 40
Initialization converged: True


In [22]:
cluster={}
for i in range(10):
    cluster[i]=[]

In [23]:
for i in range(len(labels)):
    cluster[labels[i]].append((fashion_X[i],fashion_y[i]))

In [25]:
purity,gini,confusion_matrix  = performance_calculator(cluster,10)

In [26]:
print "Purity: ",purity
print "Gini: ",gini
print "Confusion Matrix: ",display(pd.DataFrame(confusion_matrix))

Purity:  0.451342857143
Gini:  0.652456294104
Confusion Matrix: 

,0,1,2,3,4,5,6,7,8,9
0,59,1,36,12,2,301,89,0,70,13
1,1498,403,2667,3675,3596,9,2260,0,794,13
2,4293,24,1398,101,1286,0,2069,0,240,1
3,490,11,224,33,102,0,628,0,510,0
4,317,54,2623,115,1855,5,1754,0,791,5
5,0,0,0,0,0,1163,0,34,11,2495
6,46,1,26,10,14,4,59,1,1645,1
7,0,0,0,0,0,4353,0,6598,15,1688
8,34,1,17,4,10,1163,47,367,2831,2784
9,263,6505,9,3050,135,2,94,0,93,0


 None
